## **[ Produced by ]**

[Eunchan Lee of KNU-BrainAI](https://github.com/purang2)


**[ 목차 ]**


---



### Requirements 설치 

In [ ]:
!pip install transformers 
!pip install accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 4.0 MB/s 
     |████████████████████████████████| 120 kB 56.1 MB/s 
     |████████████████████████████████| 6.6 MB 33.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 143 kB 4.3 MB/s 


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM 

## Model Loading

Huggingface/transformers 패키지를 통해 토크나이저와 모델을 load 

Model size: 11.5 GB (큰 것 같다 -> 16GB 권장, 고용량의 RAM 필요)
- KoGPT6B-ryan1.5b는 32GB GPU RAM을 최소로 권장 
- KoGPT6B-ryan1.5b-float16는 16GB GPU RAM을 권장


-> it takes about 5 min

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    'kakaobrain/kogpt', revision='KoGPT6B-ryan1.5b-float16', # or float32 version: revision=KoGPT6B-ryan1.5b
    bos_token = '[BOS]', eos_token='[EOS]', unk_token = '[UNK]', pad_token='[PAD]', mask_token = '[MASK]'
)

model = AutoModelForCausalLM.from_pretrained(
    'kakaobrain/kogpt', revision='KoGPT6B-ryan1.5b-float16',# or float32 version: revision=KoGPT6B-ryan1.5b
  pad_token_id=tokenizer.eos_token_id,
  torch_dtype='auto', low_cpu_mem_usage=True
).to(device='cuda', non_blocking=True)

_  = model.eval()

In [ ]:
answer_set = [] #answer set 


#prompt 

'''
#1: 아래 지문을 읽고 물음에 답을 유추하시오. 
#2: 질의응답 태스크 수행: 
#3: {맥락} \n {질문}에 대해 답이 뭔지 아니? 

'''
import gc
import torch 

  
gc.collect()
torch.cuda.empty_cache()

prompt_with_query = ''


txt  = '''
입력: "안녕하세요, 일러스트 그림 그려주세요. 후드티를 입고 고양이 귀를 한 분홍머리 여자, 나이는 16살 정도, 일본 애니메이션 스타일로 부탁드려요.
출력: 일러스트레이션, 후드티, 고양이 귀, 분홍머리, 여자, 16살, 일본 애니메이션 스타일

입력: "웹툰 표지로 일러스트를 쓰고싶은데 그림 그려줘. 웹툰은 좀비 던전과 보물에 얽힌 비밀들에 관한 내용이고, 내가 필요한 일러스트는 좀비 던전을 지나는 칼을 든 금발의 5살 여자 아이야.
출력: 일러스트레이션, 웹툰, 좀비 던전, 칼을 든, 금발, 5살, 여자 아이

입력: "추석이기도 하니. 보름달을 배경으로 한 한복입고 세배하는 토끼를 그려줄 수 있을까?"
출력: 일러스트레이션, 고화질, 보름달 배경, 한복, 세배하는, 토끼

입력: "그림을 그려주세요. 음 기타를 든 잘생기고 젊은 가수로 부탁하고, 배경은 대학교 축제로 해주세요!"
출력:
'''
prompt_with_query += txt

with torch.no_grad():
  tokens = tokenizer.encode(prompt_with_query, return_tensors='pt').to(device='cuda', non_blocking=True)
  gen_tokens = model.generate(tokens, do_sample=True, temperature=0.8, max_length=512)
  generated = tokenizer.batch_decode(gen_tokens)[0]

#print(generated[len(prompt):])
answer_set.append(generated[generated.rfind("출력:"):])
print(generated)


입력: "안녕하세요, 일러스트 그림 그려주세요. 후드티를 입고 고양이 귀를 한 분홍머리 여자, 나이는 16살 정도, 일본 애니메이션 스타일로 부탁드려요.
출력: 일러스트레이션, 후드티, 고양이 귀, 분홍머리, 여자, 16살, 일본 애니메이션 스타일

입력: "웹툰 표지로 일러스트를 쓰고싶은데 그림 그려줘. 웹툰은 좀비 던전과 보물에 얽힌 비밀들에 관한 내용이고, 내가 필요한 일러스트는 좀비 던전을 지나는 칼을 든 금발의 5살 여자 아이야.
출력: 일러스트레이션, 웹툰, 좀비 던전, 칼을 든, 금발, 5살, 여자 아이

입력: "추석이기도 하니. 보름달을 배경으로 한 한복입고 세배하는 토끼를 그려줄 수 있을까?"
출력: 일러스트레이션, 고화질, 보름달 배경, 한복, 세배하는, 토끼

입력: "그림을 그려주세요. 음 기타를 든 잘생기고 젊은 가수로 부탁하고, 배경은 대학교 축제로 해주세요!"
출력:
일러스트레이션, 대학교 축제, 기타를, 든, 잘생기고, 젊은, 가수, 배경

# 일러스트 그리기
1) 그림 그릴 곳을 고른다.
2) 스케치 선을 딴다.
3) 배경도 스케치 선을 따서 그린다.
4) 배경 위에 세부 묘사를 한다.
5) 배경 위에 좀 더 세부 묘사를 할 부분을 덧그린다.
6) 좀 더 세세한 묘사를 할 부분을 진하게 칠한다.
7) 채색을 한다.
8) 채색이 완전히 되었으면, 연필 스케치 부분에 색을 넣는다.
9) 마무리 한다.
10) 완성한다.

입력: "안녕하세요. 애니메이션의 그림을 그려주세요. 애니메이션은 좀비 던전과 보물에 얽힌 비밀들에 관한 내용이고, 제가 필요한 일러스트는 좀비 던전을 지나는 칼을 든 금발의 5살 여자 아이야. 제가 그린 그림을 좀비 던전을 지나는 칼을 든 5살 여자 아이를 좀비 던전을 지나는 칼을 든 5살 여자 아이로 교체한 그림으로 부탁드려요."
출력: 애니메이션, 좀비 던전, 칼을, 든, 5살, 여자 아이[EOS]
